# 데이터 다운로드
https://dacon.io/competitions/official/235740/data

In [1]:
import zipfile

In [2]:
path_to_zip_file = "./data/data.zip"
directory_to_extract_to = "./data"

with zipfile.ZipFile(path_to_zip_file) as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

※ 데이터에 대한 자세한 설명은 토론 탭 데이터 및 대회 규칙 상세 설명 참조
https://dacon.io/competitions/official/235740/data





1. train_x_df.csv (10942020, 12)

임의의 시점 부터 2021년 2월 28일까지 10가지 종류의 암호화폐의 분단위 가격정보를 가공한 데이터



2. train_y_df.csv (951480, 12)

임의의 시점 부터 2021년 2월 28일까지 10가지 종류의 암호화폐의 분단위 가격정보를 가공한 데이터



3. test_x_df.csv (1048800, 12)

2021년 3월 1일 부터 임의의 시점까지 9가지 종류의 암호화폐의 분단위 가격정보를 가공한 데이터

이 중 일부 데이터를 public 점수를 계산하고, 다른 일부 데이터를 private 점수 계산하는데 사용합니다.



4. sample_submission.csv  (760, 3)

 test_x_df에는 760가지 sample_id가 존재하고 각 sample_id는 1380분 동안의 가격 변동 정보를 포함합니다.
buy_quantity : 현재 보유량 대비 샘플의 매수량 (0 ~ 1)
sell_time : 샘플의 매도 시각. (0~ 119분 사이의 정수)






[컬럼 설명]

sample_id     : 개별 샘플의 인덱스
time               : x_df는 0분 ~ 1379분, y_df는 0분 ~ 119분의 값을 갖습니다. 동일한 샘플 내 시간 정보
coin_index    : 10가지 종류의 코인에 대한 비식별화 인덱스 (0 ~9)
open              : open price
high               : high price
low                : low price
close             : close price
volume          : 거래량
quote_av       : quote asset volume
trades            : 거래 건 수
tb_base_av   : taker buy base asset volume 
tb_quote_av  : taker buy quote asset volume

In [49]:
import pandas as pd
import joblib
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [8]:
DATA_PATH = "./data"
PATH_TO_SAVE_MODEL = "./model"

In [6]:
train_x = pd.read_csv(DATA_PATH  + "/train_x_df.csv")
train_y = pd.read_csv(DATA_PATH  + "/train_y_df.csv")

In [7]:
train_x.shape, train_y.shape, test_x.shape

((10942020, 12), (951480, 12), (1048800, 12))

In [39]:
# 2차원 데이터를 3차원 데이터(sample_id, time, open)로 변환
x_train_open = pd.crosstab(index=train_x['sample_id'], columns=train_x['time'], values=train_x['open'], aggfunc='sum')
y_train_open = pd.crosstab(index=train_y['sample_id'], columns=train_y['time'], values=train_y['open'], aggfunc='sum')

In [50]:
y_train_open.iloc[:, :15].head()

time,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
sample_id,,,,,,,,,,,,,,,
0,1.000326,1.001342,1.000218,1.000689,1.001595,1.001632,1.002792,1.005620,1.007252,1.006889,1.007397,1.003372,1.003952,1.004532,1.004423
1,1.000655,1.001309,1.000561,1.000655,1.000280,1.000468,0.999719,0.999906,0.999719,0.999065,0.998504,0.999532,1.000842,1.002432,1.002525
2,1.000411,1.000971,1.001411,1.001747,1.001231,1.001031,1.000789,1.000787,1.000789,1.000715,1.000076,0.999827,0.999469,0.999626,0.999199
3,0.998480,0.996960,0.997129,0.995947,0.993075,0.992907,0.997467,0.998142,0.996453,0.997804,0.996960,0.996791,0.996791,0.995947,0.998480
4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.003677,1.003677,1.003677,1.003677


In [47]:
model_lgbm = MultiOutputRegressor(LGBMRegressor())
model_lgbm.fit(x_train_open, y_train_open.iloc[:, :16])

MultiOutputRegressor(estimator=LGBMRegressor())

In [48]:
joblib.dump(model_lgbm, './model/model_lgbm.joblib')

['./model/model_lgbm.joblib']

In [51]:
model_linear = LinearRegression()
model_linear.fit(x_train_open, y_train_open.iloc[:, :15])

LinearRegression()

In [54]:
x_train_open.head()

time,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
sample_id,,,,,,,,,,,,,,,,,,,,,
0,0.993147,0.993256,0.992748,0.994779,0.994561,0.995250,0.995033,1.000508,1.001668,0.999928,...,1.007759,1.010225,1.008883,1.004351,1.003916,1.003807,1.003336,1.000725,0.997136,1.0
1,1.045824,1.046666,1.046853,1.045918,1.046292,1.044235,1.039091,1.033293,1.034228,1.033480,...,1.001122,1.001029,0.999345,0.999158,0.999719,0.999439,0.997288,0.998691,1.000374,1.0
2,1.003717,1.003620,1.002736,1.003788,1.003119,1.001574,1.001822,1.001390,0.998011,1.001095,...,1.001095,1.001165,1.000907,1.000744,0.999618,0.999595,0.999602,0.999696,0.999860,1.0
3,1.034791,1.034623,1.034116,1.034623,1.034791,1.034623,1.034623,1.034623,1.034623,1.034285,...,1.000338,1.001013,1.000676,1.000676,1.000338,1.000169,1.000507,1.000000,0.999324,1.0
4,1.180147,1.176471,1.176471,1.180147,1.172794,1.165441,1.165441,1.165441,1.165441,1.172794,...,0.996324,0.996324,0.996324,0.996324,0.996324,0.996324,0.996324,0.996324,0.996324,1.0


In [53]:
model_linear.predict(x_train_open)

array([[1.00027527, 1.00076566, 1.00172091, ..., 0.99779852, 0.99824813,
        0.99904375],
       [1.00043056, 1.0003413 , 1.00017959, ..., 0.99863656, 0.99899733,
        0.99873294],
       [0.99974154, 1.00027284, 1.00014621, ..., 0.99931637, 0.99883423,
        0.99858084],
       ...,
       [0.99982817, 1.00026777, 1.00004864, ..., 0.99920795, 0.99902235,
        0.99874042],
       [0.9993928 , 0.99940276, 0.99941752, ..., 0.99389853, 0.9934723 ,
        0.99337243],
       [0.99999103, 1.00016277, 1.0003337 , ..., 0.99942325, 0.99922075,
        0.99907361]])

In [52]:
joblib.dump(model_linear, './model/model_linear.joblib')

['./model/model_linear.joblib']